<a href="https://colab.research.google.com/github/alexeiveselov92/coding/blob/master/%D0%92%D1%8B%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B0%20%D0%BF%D0%BE%D1%81%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D0%B9%20%D0%B8%D0%B7%20%D1%8F%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%20%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

#достаем TOKEN из OAuth Yandex зарегистрировав приложение (https://oauth.yandex.ru/)
id_with_token = '0ed90KAjgf12391a26dd' # ID специально изменен чтобы по нему нельзя было зайти
#формируем ссылку, по которой переходим и разрешаем доступ, после чего получаем токен
url_create_token = 'https://oauth.yandex.ru/authorize?response_type=token&client_id=' + id_with_token
print(url_create_token)

https://oauth.yandex.ru/authorize?response_type=token&client_id=0ed90996761840a090efadabe91a26dd


In [2]:
#вставляем полученный ТОКЕН
TOKEN = 'AgAAAAAaq1gYAAYhI31tMIqyLk_2jN7hjBnuT2E'

#указываем параметры запроса
COUNTER_ID = 45008704
DATE1 = '2018-03-01'
DATE2 = '2019-05-31'
SOURCE = 'visits'
FIELDS = ['ym:s:visitID', 'ym:s:dateTime', 'ym:s:isNewUser','ym:s:visitDuration',
                 'ym:s:startURL', 'ym:s:clientID', 'ym:s:lastTrafficSource']

#формируем параметры запроса
PARAM = {
        'date1': DATE1,
        'date2':DATE2,
        'source':SOURCE,
        'fields':','.join(FIELDS)
}

#запрос evaluate (для понимание возможности выгрузки данных)
URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequests/evaluate?'\
        .format(counter_id=COUNTER_ID)

r = requests.get(URL,headers={'Authorization': 'OAuth {0}'.format(TOKEN)},params=PARAM)       
print(r.text)

{"log_request_evaluation":{"possible":true,"max_possible_day_quantity":6744368}}


In [3]:
data = json.loads(r.text)
#Можно ли выполнить запрос получения данных из Logs API Яндекс.Метрики по данному запросу?
data['log_request_evaluation']['possible']

True

In [4]:
#создадим запрос на отчет методом logrequests
#параметры метода logrequests такие же, как у метода evaluate
URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequests?'\
        .format(counter_id=COUNTER_ID)
r = requests.post(URL,headers={'Authorization': 'OAuth {0}'.format(TOKEN)},params=PARAM)
data = json.loads(r.text)
print(data)     
#сохраним id запроса в request_id - он пригодится в дальнейшем для проверки статуса готовности данных
request_id = data['log_request']['request_id']   

{'log_request': {'request_id': 8330416, 'counter_id': 45008704, 'source': 'visits', 'date1': '2018-03-01', 'date2': '2019-05-31', 'fields': ['ym:s:visitID', 'ym:s:dateTime', 'ym:s:isNewUser', 'ym:s:visitDuration', 'ym:s:startURL', 'ym:s:clientID', 'ym:s:lastTrafficSource'], 'status': 'created', 'attribution': 'LAST'}}


In [5]:
#проверим создан ли отчёт циклом while
import time
status = 'created'
while status == 'created':
        time.sleep(60)
        print ('trying')
        URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequest/{request_id}'\
        .format(counter_id=COUNTER_ID,request_id=request_id)
        r = requests.get(URL,headers={'Authorization': 'OAuth {0}'.format(TOKEN)})
        if r.status_code == 200:
                status = json.loads(r.text)['log_request']['status']
                print (json.dumps(json.loads(r.text)['log_request'], indent = 4))
#indent - это количество пробелов

trying
{
    "request_id": 8330416,
    "counter_id": 45008704,
    "source": "visits",
    "date1": "2018-03-01",
    "date2": "2019-05-31",
    "fields": [
        "ym:s:visitID",
        "ym:s:dateTime",
        "ym:s:isNewUser",
        "ym:s:visitDuration",
        "ym:s:startURL",
        "ym:s:clientID",
        "ym:s:lastTrafficSource"
    ],
    "status": "processed",
    "size": 528193,
    "parts": [
        {
            "part_number": 0,
            "size": 528193
        }
    ],
    "attribution": "LAST"
}


In [6]:
#полученные данные мы получаем частями, если данных много, в таком случае в parts_number мы увидим количество частей
#Сначала получим номера частей, которые есть в ответе:
parts = json.loads(r.text)['log_request']['parts']
print(parts)

[{'part_number': 0, 'size': 528193}]


In [0]:
#выше получили список словарей, где каждый содержит номер части и её размер. Чтобы пройтись циклом по номерам всех частей, сведём их в один список
parts_list = []
for part in parts:
        parts_list.append(part['part_number'])

#внутри цикла сделаем запрос для получения части данных

# Обратите внимание на конструкцию StringIO(r.text). Функция StringIO() из библиотеки io позволяет прочитать строку как файл. 
# Дело в том, что метод read_csv() принимает как аргумент только файл, а часть данных в ответе от API возвращается в виде строки. 
# StringIO(r.text) превращает строку в особый тип файла — memory file (англ. «файл в памяти»). Преобразование позволяет прочитать его методом read_csv() в датафрейме tmp_df.
# Добавим эти файлы в список tmp_dfs, чтобы затем объединить их в один датафрейм logs_df методом pd.concat()

from io import StringIO
import pandas as pd

tmp_dfs = []

for part_num in parts_list:
        URL = 'https://api-metrika.yandex.ru/management/v1/counter/{counter_id}/logrequest/{request_id}/part/{part}/download' \
                        .format(
                                counter_id=COUNTER_ID,
                                request_id=request_id,
                                part=part_num
                        )

        r = requests.get(URL,headers={'Authorization': 'OAuth {0}'.format(TOKEN)})
        if r.status_code == 200:
                tmp_df = pd.read_csv(StringIO(r.text), sep = '\t') # StringIO
                tmp_dfs.append(tmp_df)

logs_df = pd.concat(tmp_dfs)

In [0]:
#экспортируем данные в виде csv-файла вызовом метода to_csv()
logs_df.to_csv('sessions_logs.csv',index=False)

In [9]:
df = pd.read_csv('sessions_logs.csv')
df

,ym:s:visitID,ym:s:dateTime,ym:s:isNewUser,ym:s:visitDuration,ym:s:startURL,ym:s:clientID,ym:s:lastTrafficSource
0,2837904752504748809,2018-05-25 10:09:07,1,24,https://ooocsk.com/kp?pm_block=premium&pm_posi...,152723214758221731,ad
1,5491454137525076094,2018-09-19 13:57:15,1,15,https://ooocsk.com/opory-truboprovodov?pm_bloc...,1537354635558442809,ad
2,1825011232304276182,2018-04-10 16:51:05,0,39,https://ooocsk.com/,1523365898957644346,direct
3,1503074878692200205,2019-03-27 11:42:55,1,51,https://ooocsk.com/,155367617696327593,organic
4,18385994706501438966,2019-01-17 10:37:39,0,15,https://ooocsk.com/ov,1547203736419407708,organic
...,...,...,...,...,...,...,...
4685,1684863856460708633,2018-04-04 12:20:45,0,16,https://ooocsk.com/sfs-5371?pm_block=premium&p...,1522821576212913491,ad
4686,3476962089154654402,2018-06-22 15:19:17,1,1616,https://ooocsk.com/,1529669988612366325,organic
4687,3635972728684032196,2018-06-29 15:48:55,1,0,https://ooocsk.com/ost-34-10-610-93,15302765301029233047,internal
4688,1528805563507683523,2018-03-28 14:58:50,1,15,https://ooocsk.com/otvody-stalnye-gost-17375-2001,1522238331645961906,ad
